In [1]:
import sys, os
import pandas as pd
import numpy
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [8]:
def nbc(t_frac):
    df = pd.read_csv('trainingSet.csv').sample(frac=t_frac, random_state=47)
    return df

In [9]:
df = nbc(1)

In [10]:
# Print the accuracy

      gender  age  age_o  race  race_o  samerace  importance_same_race  \
5348       1    1      0     2       2         1                     0   
1225       0    0      0     0       2         0                     1   
4424       0    1      0     0       2         0                     4   
1338       1    0      1     2       2         1                     1   
227        0    0      0     1       2         0                     3   
825        0    0      1     0       2         0                     0   
958        1    1      1     2       2         1                     0   
2130       1    1      1     2       2         1                     3   
4315       1    0      1     2       2         1                     1   
1790       0    0      1     2       2         1                     2   
4124       1    1      0     2       2         1                     1   
4804       1    0      0     2       0         0                     0   
3344       0    0      1     1       2